
# 감성 분석 모델

**구 성**
- 라벨: **긍정(1) / 부정(0)**
- 5단계: 데이터 로드 → 전처리 → 모델 생성 → 학습 → 추론

In [57]:
import os, re, unicodedata
from typing import List
import numpy as np
import pandas as pd

from IPython.display import display

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LSTM, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping

print("TensorFlow:", tf.__version__)


TensorFlow: 2.16.1


In [58]:
# 허용 문자 외는 공백으로 치환하는 정규식
# 주의: raw string(r"...") 안에 \s 는 한 번만 씁니다.
KOREAN_KEEP_REGEX = re.compile(r'[^0-9A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ\s\.,!?:;\-\(\)\'"]')

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    s = unicodedata.normalize("NFKC", s)
    s = KOREAN_KEEP_REGEX.sub(" ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [59]:
def combine_text_columns(df: pd.DataFrame, text_cols: List[str]) -> List[str]:
    use_cols = [c for c in text_cols if c in df.columns]
    if not use_cols:
        # '문장' 또는 '발화' 포함 컬럼 폴백
        cand = [c for c in df.columns if ("문장" in c or "발화" in c)]
        use_cols = cand[:3] if cand else [df.columns[0]]

    texts = (
        df[use_cols].fillna("")
                    .astype(str)
                    .agg(" [SEP] ".join, axis=1)
                    .map(normalize_text)
                    .tolist()
    )
    return texts

## 1) 데이터 로드

In [60]:
# 파일 경로
XLSX_PATH = "./감성대화말뭉치(최종데이터)_Training.xlsx"  # 필요 시 변경

# 라벨/텍스트 컬럼
# 기본은 '감정_대분류'를 사용하며, 없으면 '감정_분류'를 자동 폴백으로 사용
PRIMARY_LABEL_CANDIDATES = ["감정_대분류"]
TEXT_COLS = ["사람문장1", "사람문장2", "사람문장3"]

# 토크나이저/모델 하이퍼파라미터
NUM_WORDS = 30000
MAX_LEN   = 96
EMBED_DIM = 128
RNN_UNITS = 128
BATCH_SIZE = 64
EPOCHS     = 8
PATIENCE   = 2


## 3) 데이터 로드 & 전처리

In [61]:
def read_excel(xlsx_path: str) -> pd.DataFrame:
    assert os.path.exists(xlsx_path)
    return pd.read_excel(xlsx_path)

df = read_excel(XLSX_PATH)
print("Columns:", list(df.columns))
display(df.head(3))

# 라벨 컬럼 결정
label_col = None
for cand in PRIMARY_LABEL_CANDIDATES:
    if cand in df.columns:
        label_col = cand
        break
assert label_col is not None, f"라벨 컬럼({PRIMARY_LABEL_CANDIDATES})을 데이터에서 찾을 수 없습니다."
print("Using label column:", label_col)

Columns: ['Unnamed: 0', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1', '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3']


,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN


Using label column: 감정_대분류


In [62]:
# 텍스트 결합
texts = combine_text_columns(df, TEXT_COLS)

# 이진 라벨: 대분류가 '기쁨'이면 1(긍정), 그 외는 0(부정)
def to_binary_label(row) -> int:
    major = str(row[label_col]) if pd.notna(row[label_col]) else ""
    return 1 if "기쁨" in major else 0

y = df.apply(to_binary_label, axis=1).astype(int).to_numpy()

# 토크나이저 & 패딩
tok = Tokenizer(num_words=NUM_WORDS, oov_token="<unk>")
tok.fit_on_texts(texts)
X = pad_sequences(tok.texts_to_sequences(texts), maxlen=MAX_LEN, padding="post", truncating="post")

# 분할
n = len(X)
idx = np.arange(n)
np.random.seed(42); np.random.shuffle(idx)
n_test = int(n * 0.1)
n_val  = int(n * 0.1)
test_idx = idx[:n_test]
val_idx  = idx[n_test:n_test+n_val]
train_idx = idx[n_test+n_val:]

X_train, y_train = X[train_idx], y[train_idx]
X_val,   y_val   = X[val_idx],   y[val_idx]
X_test,  y_test  = X[test_idx],  y[test_idx]

X.shape, X_train.shape, X_val.shape, X_test.shape, y.mean()

((51630, 96), (41304, 96), (5163, 96), (5163, 96), 0.11865194654270773)

## 4) 모델 정의 및 생성 (BiLSTM)

In [63]:
def build_bilstm_binary(vocab_size: int, max_len: int,
                        embed_dim: int = 128, rnn_units: int = 128, dropout: float = 0.3) -> Model:
    inputs = Input(shape=(max_len,), name="input_ids")
    x = Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=max_len, name="emb")(inputs)
    x = Bidirectional(LSTM(rnn_units, return_sequences=True))(x)
    x = GlobalMaxPooling1D()(x)
    x = Dropout(dropout)(x)
    x = Dense(rnn_units, activation="relu")(x)
    x = Dropout(dropout)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

vocab_size = min(NUM_WORDS, len(tok.word_index) + 1)
model = build_bilstm_binary(vocab_size=vocab_size, max_len=MAX_LEN,
                            embed_dim=EMBED_DIM, rnn_units=RNN_UNITS)
model.summary()


/opt/anaconda3/envs/nlp_env/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)          │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ emb (Embedding)                 │ (None, 96, 128)        │     3,840,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 96, 256)        │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_5          │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,136,193 (15.78 MB)

 Trainable params: 4,136,193 (15.78 MB)

 Non-trainable params: 0 (0.00 B)

## 5) 모델 학습

In [64]:
callbacks = [EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True)]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Acc: {test_acc:.4f} | Test Loss: {test_loss:.4f}")


Epoch 1/8
646/646 ━━━━━━━━━━━━━━━━━━━━ 90s 137ms/step - accuracy: 0.9449 - loss: 0.1574 - val_accuracy: 0.9673 - val_loss: 0.0953
Epoch 2/8
646/646 ━━━━━━━━━━━━━━━━━━━━ 88s 136ms/step - accuracy: 0.9847 - loss: 0.0476 - val_accuracy: 0.9671 - val_loss: 0.0936
Epoch 3/8
646/646 ━━━━━━━━━━━━━━━━━━━━ 88s 137ms/step - accuracy: 0.9950 - loss: 0.0180 - val_accuracy: 0.9642 - val_loss: 0.1324
Epoch 4/8
646/646 ━━━━━━━━━━━━━━━━━━━━ 86s 133ms/step - accuracy: 0.9971 - loss: 0.0083 - val_accuracy: 0.9640 - val_loss: 0.1910
Test Acc: 0.9655 | Test Loss: 0.0996


## 6) 추론

In [65]:
def predict_proba(text_list: List[str], threshold: float = 0.5) -> pd.DataFrame:
    texts_norm = [normalize_text(t) for t in text_list]
    Xp = pad_sequences(tok.texts_to_sequences(texts_norm), maxlen=MAX_LEN, padding="post", truncating="post")
    probs = model.predict(Xp, verbose=0).reshape(-1)
    preds = (probs >= threshold).astype(int)
    label_map = {0: "부정", 1: "긍정"}

    df_out = pd.DataFrame({
        "입력 문장": text_list,
        "긍정 확률": probs.round(3),
        "판정": [label_map[int(lbl)] for lbl in preds]
    })
    return df_out


sentence = ["오늘 너무 화나는 일이 있었어", "기분 최고야 너무 행복해", "그냥 마음이 허하고 슬프다"]
predict_proba(sentence)

,입력 문장,긍정 확률,판정
0,오늘 너무 화나는 일이 있었어,0.351,부정
1,기분 최고야 너무 행복해,0.997,긍정
2,그냥 마음이 허하고 슬프다,0.063,부정
